<a href="https://colab.research.google.com/github/MinsooKwak/Study/blob/main/LLM/Reformer/Reformer_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Sequence Classification
- Text Classification
- Mask Language Model
- QA

등 활용 가능

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, ReformerForQuestionAnswering

In [3]:
tokenizer = AutoTokenizer.from_pretrained('google/reformer-crime-and-punishment')
model = ReformerForQuestionAnswering.from_pretrained('google/reformer-crime-and-punishment')

Some weights of ReformerForQuestionAnswering were not initialized from the model checkpoint at google/reformer-crime-and-punishment and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


컨텍스트가 방대하지 않으면 모델이 예측하기 어렵다

In [4]:
question = "What is the captial of Korea?"
text = "Korea is a country based in Asia and captial name is Seoul"

In [5]:
inputs = tokenizer(question, text, return_tensors='pt')
with torch.no_grad():
  outputs = model(**inputs)

In [6]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.1661, -0.2316,  0.1025, -0.6009, -0.1190, -0.8151,  0.4653, -0.0908,
         -0.1455, -0.3648, -0.4621, -0.6549, -0.0673, -0.5496, -0.5781,  0.2484,
          0.1362,  0.5811, -0.2626, -0.8765,  0.0948, -0.1434,  0.1087, -0.7147,
         -0.5301, -0.5204,  0.4146, -0.2345,  0.1398, -0.8495, -0.3496, -0.1810,
         -0.7655, -0.1492, -0.0192,  0.2629,  0.4971, -0.7573,  1.0077,  0.0723,
          0.2821,  0.6228, -0.6528,  0.0587,  0.0024, -0.3184, -0.3425, -0.4197,
          0.0318]]), end_logits=tensor([[ 0.1632, -0.4220, -0.1269, -0.7571,  0.1352,  0.5468,  0.5225,  0.4522,
         -0.1171, -0.3321, -0.5786, -0.3919,  0.6108, -0.5566,  0.2683, -0.6924,
          0.1543,  0.6581, -0.5375,  0.4290, -0.5346, -0.3821,  0.3118,  0.1083,
          0.2949, -0.0673,  0.2392,  0.1501,  0.5361, -0.6163,  0.5601, -0.1534,
          0.5657,  0.2238,  0.3016, -0.5319,  0.1158,  0.5672,  0.7670,  0.3140,
          0.0090, -0.122

- loss : None
- 가장 중요한 것은 start_logits, end_logits

In [7]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [8]:
answer_start_index  # 답변이 38 인덱스에서 시작된다

tensor(38)

In [9]:
answer_end_index    # 답변이 43 인덱스에서 끝난다

tensor(43)

In [10]:
predicted_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index +1]

# 정답이 정확히 Text의 어디에 있는지 알기 위해 설정
target_start_index = torch.tensor([11]) # Seoul이 11에 있으니까 해당 값을 전달하는 것 /zero shot 아니므로
target_end_index = torch.tensor([11])

In [12]:
outputs = model(**inputs,
                start_positions=target_start_index,
                end_positions=target_end_index)
loss = outputs.loss

In [13]:
loss

tensor(4.4630, grad_fn=<DivBackward0>)

- 인덱스에 따른 손실을 제공함

In [14]:
def decode_qa_output(question, context, start_logits, end_logits, tokenizer):

  start_logits = torch.tensor(start_logits)
  end_logits = torch.tensor(end_logits)

  # highest logits로 start, end indices 가져옴
  start_index = start_logits.argmax()
  end_index = end_logits.argmax()

  # answer span에 맞는 token 가져옴
  tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(context))
  answer_tokens = tokens[start_index:end_index+1]

  # Decode Answer token to text
  answer_text = tokenizer.convert_tokens_to_string(answer_tokens)

  # Special 토큰 제거, 원치 않는 문자열 제거
  answer_text = answer_text.replace('[CLS]','').replace('[SEP]','').strip()

  return answer_text

In [16]:
start_logits = outputs['start_logits']
end_logits = outputs['end_logits']

decoded_answer = decode_qa_output(question, text, start_logits, end_logits, tokenizer)

<ipython-input-14-859d602ed72c>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_logits = torch.tensor(start_logits)
<ipython-input-14-859d602ed72c>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  end_logits = torch.tensor(end_logits)


In [17]:
print(decoded_answer)

- 아무 값이 안나타나는데, start_logits, end_logits를 확인했을 때 어떤 최대 인덱스 갖지 않는 것을 볼 수 있음
- 앞에 설정한 텍스트 최대 길이에 미치지 못함

In [18]:
start_logits

tensor([[-0.1661, -0.2316,  0.1025, -0.6009, -0.1190, -0.8151,  0.4653, -0.0908,
         -0.1455, -0.3648, -0.4621, -0.6549, -0.0673, -0.5496, -0.5781,  0.2484,
          0.1362,  0.5811, -0.2626, -0.8765,  0.0948, -0.1434,  0.1087, -0.7147,
         -0.5301, -0.5204,  0.4146, -0.2345,  0.1398, -0.8495, -0.3496, -0.1810,
         -0.7655, -0.1492, -0.0192,  0.2629,  0.4971, -0.7573,  1.0077,  0.0723,
          0.2821,  0.6228, -0.6528,  0.0587,  0.0024, -0.3184, -0.3425, -0.4197,
          0.0318]], grad_fn=<CloneBackward0>)

In [20]:
start_logits.argmax()

tensor(38)

In [19]:
end_logits

tensor([[ 0.1632, -0.4220, -0.1269, -0.7571,  0.1352,  0.5468,  0.5225,  0.4522,
         -0.1171, -0.3321, -0.5786, -0.3919,  0.6108, -0.5566,  0.2683, -0.6924,
          0.1543,  0.6581, -0.5375,  0.4290, -0.5346, -0.3821,  0.3118,  0.1083,
          0.2949, -0.0673,  0.2392,  0.1501,  0.5361, -0.6163,  0.5601, -0.1534,
          0.5657,  0.2238,  0.3016, -0.5319,  0.1158,  0.5672,  0.7670,  0.3140,
          0.0090, -0.1221, -0.5748,  0.8040, -0.0058,  0.6002,  0.0815, -0.2164,
          0.6117]], grad_fn=<CloneBackward0>)

In [21]:
end_logits.argmax()

tensor(43)